<a href="https://colab.research.google.com/github/ShaunakSen/ML_Deployment/blob/master/Benchmarking_XGBoost_with_GPU_and_HummingBird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Benchmarking XGBoost o GPU

> Based on the YouTube tutorial by AIEngineering: https://www.youtube.com/watch?v=X7cKC6GgyxY

---



In [1]:
import xgboost as xgb
xgb.__version__

'0.90'

In [0]:
from __future__ import print_function
import sys, tempfile, urllib, os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

Import a dataset called "covertype"

In [0]:
from sklearn.datasets import fetch_openml

covtyp = fetch_openml(name="covertype", version=4)

In [5]:
print (covtyp.DESCR)

**Author**: Jock A. Blackard, Dr. Denis J. Dean, Dr. Charles W. Anderson  
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Covertype) - 1998  

This is the original version of the famous covertype dataset in ARFF format. 

**Covertype**  
Predicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System &#40;RIS&#41; data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data. Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types). 

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are 

In [9]:
print (covtyp.feature_names, len(covtyp.feature_names))

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40'] 54


In [8]:
covtyp.data.shape

(581012, 54)

In [11]:
covtyp.target.shape

(581012,)

In [12]:
print ("Unique classes:", np.unique(covtyp.target))

Unique classes: ['1' '2' '3' '4' '5' '6' '7']


As we can see the data size is quite substantiable

It is a **multi-class classification problem**

In [14]:
!nvidia-smi

Sat Jun 13 08:44:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Ok, so we have a Tesla K80 GPU with 10 GB memory

## Benchmarking with Hummingbird

> Based on the YouTube tutorial by AIEngineering: https://www.youtube.com/watch?v=XWD4F7LNvNE

---



In [0]:
!nvidia-smi

Sat Jun 13 07:46:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Ok, so we have a Tesla T4 GPU enabled with 15GB memeory